In [14]:
from openai import OpenAI
import os
import pandas as pd
from tqdm.auto import tqdm
import json
import uuid

In [2]:
client = OpenAI(api_key=os.environ["IA"])

In [3]:
test = pd.read_json("./documents.json").to_dict(orient="records")

In [4]:
chunk_list = []
ids = []

In [5]:
for i in test:
    if i["length"] > 1700:
        chunk_list.append({"source": i["source"], "content": i["content"], "headline":i["headline"], "length":i["length"]})
        ids.append(i["id"])

In [6]:
len(ids), ids[0]

(105, '8c1c6484-7c57-4e2d-93e6-84c5af76e049')

In [7]:
filtered_list = [record for record in test if record['id'] not in ids]

In [8]:
len(filtered_list) + len(chunk_list) == len(test)

True

In [11]:
def fetch_knowledge_chunks(Chunk):
    content = Chunk["content"]
    source = Chunk["source"]
    headline = Chunk["headline"]
    prompt = f"""Task: Extract Knowledge Chunks
    Please extract knowledge chunks from the following content.
    Return the extracted knowledge chunks as a JSON object or array (even if there is a 1 chunk), 
    ensuring that each chunk approximately 1000 character (not strict rule).
    If text is approximately 2000 character extract more than 1 chunk.
    Text is includes markdown format. keep link, list, etc. Use the format: 
    {{"chunks": "chunk"}}
    
    
    Text:

    {content}
    """

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": "You are an assistant that takes apart a piece of text into semantic chunks to be used in a RAG system.",
            },
            {"role": "user", "content": prompt},
        ],
        stream=False,
    )

    res = json.loads(completion.choices[0].message.content)

    return res, source, headline

In [12]:
len(chunk_list[0]["content"])

1745

In [13]:
chundked_list=[]

In [15]:
for chunk in tqdm(chunk_list):
    res, source, headline = fetch_knowledge_chunks(chunk)
    for r in res["chunks"]:
        if len(r)==1:
            r = r["chunk"] or r["text"]
            chundked_list.append({"source":source, "headline":headline, "content":r, "length": len(r), "id": str(uuid.uuid4())})
        else:
            chundked_list.append({"source":source, "headline":headline, "content":r, "length": len(r), "id": str(uuid.uuid4())})

  0%|          | 0/105 [00:00<?, ?it/s]

In [16]:
len(chundked_list)

344

In [18]:
filtered_list[1]

{'source': 'https://www.reddit.com/r/germany/wiki/citizenship',
 'content': '*By u/staplehill*\nGerman citizenship is [not based on where you are born](https://en.wikipedia.org/wiki/Jus_soli). You get German citizenship either through naturalization after living in Germany for a number of years, or you inherit it from a parent. German citizenship *can* be passed down through multiple generations who live abroad and have no idea that they are actually German citizens. But one can also lose German citizenship just as unknowingly, for example by voluntarily getting the citizenship of another country (1914-today), or due to sex-discriminatory laws that in the past often allowed only the father to pass on German citizenship to a child. If German citizenship got lost due to sex discrimination then there may now be a pathway to restore it - but often not if it was lost for other reasons.\nDo you have a German ancestor and want to know if you can get a German passport? In order to find out, we

In [19]:
chundked_list[1]

{'source': 'https://www.reddit.com/r/germany/wiki/american-dream',
 'headline': 'The American Dream',
 'content': 'This means: The American Dream is doing fine - in Europe and Canada. Regarding daughters, they have found that their income is in all countries determined by the factor if she marries someone, who she marries if she has children and how many. The jury is still out about how to evaluate this result. Some people argue that it is not her own achievement if her husband makes a lot of money and that it is also not an achievement to not have children. Others argue that it is an achievement to marry a man with a high earning potential and use contraceptives correctly.',
 'length': 567,
 'id': 'd4bee544-64bc-41d6-ab52-ca3ac48cfc8c'}

In [20]:
doc_chunked = filtered_list + chundked_list
len(doc_chunked)

614

In [21]:
with open('doc_chunked.json', 'wt') as f_out:
    json.dump(doc_chunked, f_out, indent=2)